In [1]:
from kedro.framework.session import KedroSession
from kedro.framework.startup import bootstrap_project
from pathlib import Path

project_path = Path.cwd()
bootstrap_project(project_path)

session = KedroSession.create(project_path=project_path)
context = session.load_context()

catalog = context.catalog

[01/01/26 03:50:13] INFO     Using 'conf\logging.yml' as logging configuration. You can change this ]8;id=754970;file://C:\Users\V\miniforge3\envs\nn\Lib\site-packages\kedro\framework\project\__init__.py\__init__.py]8;;\:]8;id=270974;file://C:\Users\V\miniforge3\envs\nn\Lib\site-packages\kedro\framework\project\__init__.py#269\269]8;;\
                             by setting the KEDRO_LOGGING_CONFIG environment variable accordingly.                 

[01/01/26 03:50:16] WARNING  C:\Users\V\miniforge3\envs\nn\Lib\site-packages\mlflow\pyfunc\utils\da ]8;id=861391;file://C:\Users\V\miniforge3\envs\nn\Lib\warnings.py\warnings.py]8;;\:]8;id=570323;file://C:\Users\V\miniforge3\envs\nn\Lib\warnings.py#110\110]8;;\
                             ta_validation.py:186: UserWarning: [33mAdd type hints to the `predict`                
                             method to enable data validation and automatic signature inference                    
                             during model logging. Check                                                           
                             https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage                
                             -in-pythonmodel for more details.[0m                                                  
                               color_warning(                                                                      
                                                                                                                   

[01/01/26 03:50:17] INFO     Registering new custom resolver: 'km.random_name'                    ]8;id=66054;file://C:\Users\V\miniforge3\envs\nn\Lib\site-packages\kedro_mlflow\framework\hooks\mlflow_hook.py\mlflow_hook.py]8;;\:]8;id=115643;file://C:\Users\V\miniforge3\envs\nn\Lib\site-packages\kedro_mlflow\framework\hooks\mlflow_hook.py#65\65]8;;\

                    INFO     The 'tracking_uri' key in mlflow.yml is relative            ]8;id=4164;file://C:\Users\V\miniforge3\envs\nn\Lib\site-packages\kedro_mlflow\config\kedro_mlflow_config.py\kedro_mlflow_config.py]8;;\:]8;id=958636;file://C:\Users\V\miniforge3\envs\nn\Lib\site-packages\kedro_mlflow\config\kedro_mlflow_config.py#294\294]8;;\
                             ('server.mlflow_(tracking|registry)_uri = mlruns'). It is                             
                             converted to a valid uri:                                                             
                             'file:///C:/Users/V/Desktop/Kedro-pytorch-base/disease-dete                           
                             ction/mlruns'                                                                         

                    INFO     Registering new custom resolver: 'km.random_name'                    ]8;id=98486;file://C:\Users\V\miniforge3\envs\nn\Lib\site-packages\kedro_mlflow\framework\hooks\mlflow_hook.py\mlflow_hook.py]8;;\:]8;id=810605;file://C:\Users\V\miniforge3\envs\nn\Lib\site-packages\kedro_mlflow\framework\hooks\mlflow_hook.py#65\65]8;;\

In [3]:
train_ds = catalog.load("train_augmented_ds")  
# or "train_augmented_ds" if you already created it

len(train_ds), list(train_ds.keys())[:5]

[01/01/26 03:51:17] INFO     Loading data from train_augmented_ds (PartitionedDataset)...      ]8;id=495969;file://C:\Users\V\miniforge3\envs\nn\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=434757;file://C:\Users\V\miniforge3\envs\nn\Lib\site-packages\kedro\io\data_catalog.py#1048\1048]8;;\


(
    4990,
    [
        'fold_0/Healthy/healthy (1).jpeg',
        'fold_0/Healthy/healthy (1).jpeg_aug',
        'fold_0/Healthy/healthy (10).jpeg',
        'fold_0/Healthy/healthy (10).jpeg_aug',
        'fold_0/Healthy/healthy (103).jpeg'
    ]
)

In [4]:
import fiftyone as fo
import numpy as np
import tempfile
import cv2
from pathlib import Path

fo_dataset = fo.Dataset("train_augmented_ds")

tmp_dir = Path(tempfile.mkdtemp())

for key, loader in train_ds.items():
    img = loader()
    if img is None:
        continue

    class_name = key.split("/")[0]

    out_path = tmp_dir / f"{key.replace('/', '_')}.jpg"
    out_path.parent.mkdir(parents=True, exist_ok=True)

    cv2.imwrite(str(out_path), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

    fo_dataset.add_sample(
        fo.Sample(
            filepath=str(out_path),
            ground_truth=fo.Classification(label=class_name)
        )
    )

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 import fiftyone as fo                                                                       │
│    2 import numpy as np                                                                          │
│    3 import tempfile                                                                             │
│    4 import cv2                                                                                  │
│                                                                                                  │
│ C:\Users\V\miniforge3\envs\nn\Lib\site-packages\fiftyone\__init__.py:31 in <module>              │
│                                                                                                  │
│ C:\Users\V\miniforge3\envs\nn\Lib\site-packages\fiftyone\__public__.py:150 in <module>           │
│                                                                                                  │
│ C:\Users\V\miniforge3\envs\nn\Lib\site-packages\fiftyone\core\plots\__init__.py:25 in <module>   │
│                                                                                                  │
│ C:\Users\V\miniforge3\envs\nn\Lib\site-packages\fiftyone\core\plots\views.py:21 in <module>      │
│                                                                                                  │
│ C:\Users\V\miniforge3\envs\nn\Lib\site-packages\fiftyone\core\plots\plotly.py:27 in <module>     │
│                                                                                                  │
│ C:\Users\V\miniforge3\envs\nn\Lib\site-packages\fiftyone\core\plots\utils.py:12 in <module>      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModuleNotFoundError: No module named 'sklearn'

In [ ]:
len(fo_dataset)

In [ ]:
fo_dataset.first().filepath


In [ ]:
model = foz.load_zoo_model(
    "mobilenet-v2-imagenet-torch",
    embeddings=True,
)


In [ ]:
import sklearn
import fiftyone

print("sklearn:", sklearn.__version__)
print("fiftyone:", fiftyone.__version__)

In [ ]:
import fiftyone.brain as fob
fob.compute_similarity(
    fo_dataset,
    embeddings="embeddings",
    metric="cosine",
)

In [ ]:
from fiftyone import ViewField as F

view = fo_dataset.match(F("similarity") > 0.92)
session = fo.launch_app(view)